<a href="https://colab.research.google.com/github/cadyngo/EAS-Math-for-AI/blob/main/MatrixFactorization_TODO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Matrix Factorization Methods

Now that we have covered the fundamentals of matrix multiplication, let us discuss some important methods in matrix factorization. Firstly, what is matrix factorization? Matrix factorization is a technique in linear algebra that involves breaking down a matrix into a product of other matrices, similar to how factorization works for scalars.

## Exercise 1: LU Decomposition

LU decomposition with partial pivoting implies the following condition. For a square, nonsingular matrix A $∈\mathbb{R}^{n\times n}$, LU with partial pivoting states:

$$PA=LU$$

where $P$ is the permutation matrix (representing row swaps), $L$ is the unit lower triangular matrix ($diag(L) = 1$), and $U$ is the upper triangular matrix.

**We obtain this by performing Gaussian elimination:**

* at step $k$, we select a pivot (usually the entry of largest magnitude in column $k$ at or below the diagonal) and optionally swap rows to move it into the $(k, k)$ position (this updates our permutation matrix $P$)

* We then subtract multiples of the pivot row from rows below to create zeros beneath the pivot

* The subtraction factors for each row $r$ in column $k$ become the subdiagonal entries of $L$ at entry $(r, k)$

* Furthermore, what remains after all gaussian transformations on our original matrix $A$ is $U$

**Why is this important for machine learning?**

There are many instances where this factorization can be useful.
We may want to solve a system $Ax=b$ for many different $b$, a task that is relatively common in machine learning - multi-output regression, for example. Performing LU decomposition once and performing the two triangular solves with $L$ and $U$ is generally computationally cheaper than solving the original system $Ax=b$.

**Task:** Implement a function `lu3x3(A)` that identifies and returns $L$ and $U$ for a given $A$. Consider pivoting but don't find the permutation matrix $P$.

In [ ]:
import numpy as np
def lu3x3(A):
  # TODO: Write solution here

  return L, U

# Identify L, U
A = np.array([[1, 1, 1], [0, 1, 1], [1, 0, 1]])
L, U = lu3x3(A)

print("A:")
print(A)

print("\nL:")
print(L)
print("\nU:")
print(U)

# Verify that our solution works for an arbitrary
b = np.array([4, 3, 2])

x_org = np.linalg.solve(A, b)
x_LU = np.linalg.solve(U, np.linalg.solve(L, b))

print("\nOriginal solution:")
print(x_org)

print("\nLU solution:")
print(x_LU)


A:
[[1 1 1]
 [0 1 1]
 [1 0 1]]

L:
[[ 1.  0.  0.]
 [ 0.  1.  0.]
 [ 1. -1.  1.]]

U:
[[1 1 1]
 [0 1 1]
 [0 0 1]]

Original solution:
[1. 2. 1.]

LU solution:
[1. 2. 1.]


## Exercise 2: Singular Value Decomposition (SVD)

Singular Value Decomposition (SVD) is a technique to factorize a matrix into 3 different sub-matrices. The specific conditions of SVD are as follows:

For any real matrix $A\in\mathbb{R}^{m\times n}$ there exist orthogonal matrices $U\in\mathbb{R}^{m\times m}$ and $V\in\mathbb{R}^{n\times n}$ and a diagonal (rectangular) matrix $\Sigma\in\mathbb{R}^{m\times n}$ with nonnegative diagonal entries $\sigma_1\ge\cdots\ge \sigma_r>0$ (where $r=\mathrm{rank}(A)$) such that:

$$A=UΣV^T,\quad Σ=\begin{bmatrix} (diag(σ_1, ...,σ_r)) & 0 \\ 0 & 0 \end{bmatrix}$$

**Pseudoinverse**
The Moore–Penrose pseudoinverse of $A$ is defined via its SVD.

If $A=U\Sigma V^T$ with $\Sigma=\operatorname{diag}(\sigma_1,\ldots,\sigma_r,0,\ldots)$, then the pseudoinverse is defined as follows $$A^+=VΣ^+U^T$$ where $\Sigma^+\in\mathbb{R}^{n\times m}$ is formed by taking reciprocals of the nonzero singular values and transposing the rectangular shape: $\Sigma^+=\operatorname{diag}(\frac{1}{\sigma_1},\ldots,\frac{1}{\sigma_r},0,\ldots)$.

With this, the least–squares solution to $Ax\approx b$ that minimizes $|Ax-b|_2$ (and among all minimizers, has minimum $|x|_2$) is $x^*=A^+b.$

*Useful special cases:*

* if $A$ has full column rank ($m\ge n$, $\operatorname{rank}(A)=n$) then $(A^TA)^{-1}A^T=A^+$

* if $A$ has full row rank ($m\le n$, $\operatorname{rank}(A)=m$) then $A^T(AA^T)^{-1}=A^+$
  * Geometrically, $AA^+$ projects onto $\operatorname{col}(A)$ and $A^+A$ projects onto $\operatorname{row}(A)$

**Why is this important in machine learning?**

SVD underlies several core ideas:

* *Dimensionality reduction (PCA)*:
  * For a centered data matrix $X\in\mathbb{R}^{m\times n}$ (rows = samples, columns = features), the SVD $X=UΣV^T$ yields principal directions in the columns of $V$
  * Truncating to the top $k$ singular values gives a low-dimensional representation $X≈U_kΣ_kV_k^T$ where $U_k\in\mathbb{R}^{m\times k}$, $V_k\in\mathbb{R}^{n\times k}$

* *Least squares and stability:*
  * Linear regression normal equations can be written with the pseudoinverse: for targets $y$, the minimal-norm LS solution is $\hat{w} = X^+y$
  * Viewing this through the SVD shows which directions (small $\sigma_i$) are ill-conditioned and amplify noise

**Task:** Write `pinv_svd` to compute $A^+=VΣ^+U^T$ and solve on an ill-conditioned system $x^*=A^+b$, and compare this to normal equations.

*Note:* To find the SVD, use `np.linalg.svd`, as it is difficult to compute by hand and thus algorithmically

In [ ]:
def pinv_svd(A, tol=1e-12):
  # TODO: Implement SVD inverse here
  return

# EXAMPLE: Build an ill-conditioned tall system
np.random.seed(0)
m, n = 60, 15
A = np.random.randn(m, n)
A[:, 0] = A[:, 1] + 1e-8*np.random.randn(m)

# Find ground truth with a little noise
x_true = np.random.randn(n)
b = A @ x_true + 1e-3*np.random.randn(m)

# Solve - SVD
x_pinv = pinv_svd(A) @ b

# Solve - normal equations/lstsq
try:
    x_ne = np.linalg.solve(A.T @ A, A.T @ b)
except np.linalg.LinAlgError:
    x_ne = np.linalg.lstsq(A, b, rcond=None)[0]

# Compare errors
rel = lambda x: np.linalg.norm(x - x_true)/np.linalg.norm(x_true)
print("||A x_pinv - b||₂ =", np.linalg.norm(A @ x_pinv - b))
print("||A x_ne   - b||₂ =", np.linalg.norm(A @ x_ne   - b))


||A x_pinv - b||₂ = 0.005156966405694077
||A x_ne   - b||₂ = 0.005171705003884181


## Exercise 3: Eigenvalue Decomposition (EVD)

For a square matrix $A\in\mathbb{R}^{n\times n}$, an eigenpair $(\lambda,x\neq 0)$ satisfies $Ax=\lambda x$. If $A$ has $n$ linearly independent eigenvectors (i.e., is diagonalizable), then there exist an invertible $X$ and a diagonal $\Lambda=\mathrm{diag}(\lambda_1,\dots,\lambda_n)$ such that

$$A=XΛX^{-1}\quad\text{and}\quad AX=ΛX$$

A particularly important special case is the real symmetric (or complex Hermitian) case, where the spectral theorem guarantees the following orthogonal diagonalization:

$$A=A^T ⇒ A=QΛQ^T, \quad Q^TQ=I, Λ∈\mathbb{R}^{n\times n} diagonal$$

*When does $A=X\Lambda X^{-1}$ exist?*

* $A$ is diagonalizable over $\mathbb{C}$ iff the direct sum of eigenspaces has dimension $n$ (equivalently, the minimal polynomial has no repeated root). If not, $A$ is defective and has a Jordan form instead of a diagonal one

* Over $\mathbb{R}$, a real $X\Lambda X^{-1}$ requires a full set of real eigenvectors

**Useful Special Cases for Finding the EVD computationally**

* Symmetric/Hermitian: $A=Q\Lambda Q^\top$ (or $Q\Lambda Q^{!*}$ in complex). Use `np.linalg.eigh` (most accurate; returns real eigenvalues and orthonormal eigenvectors).

* Normal matrices (commute with their transpose/unitary adjoint): unitarily diagonalizable; use `eigh` in the Hermitian case.

* General (possibly nonsymmetric): use `np.linalg.eig` (eigenvalues/vectors may be complex; eigenvectors need not be orthogonal).

**Why is EVD important in machine learning?**

* PCA via covariance: For centered data $X\in\mathbb{R}^{m\times d}$, the covariance $C=\frac{1}{m-1}X^\top X$ is symmetric PSD. Its EVD $C=Q\Lambda Q^\top$ gives principal directions (columns of $Q$) and explained variances (diagonal of $\Lambda$). (This matches SVD: $X=U\Sigma V^\top \Rightarrow C=V\Sigma^2V^\top$.)

* Quadratic forms & Rayleigh quotient: For unit $x$, $x^\top A x\in[\lambda_{\min},\lambda_{\max}]$ when $A$ is symmetric. This underlies variance maximization in PCA and many optimization bounds.

* Matrix functions & dynamics: If $A=Q\Lambda Q^\top$ (symmetric), then $A^k=Q\Lambda^kQ^\top$ and $e^{A}=Qe^{\Lambda}Q^\top$. This decouples linear systems and graph diffusion.

**Task**

Implement a helper function `evd_symmetric` that calls np.linalg.eigh(A) and verifies $A\approx Q\Lambda Q^\top$ and $Q^\top Q=I$.

In [ ]:
def evd_symmetric(A):
    # TODO: Implement solution here
    return

# quick test
rng = np.random.default_rng(0)
M = rng.normal(size=(6,6)); A = 0.5*(M+M.T)

# evaluate reconstruction and orthonormal Q
Q, w, ok, ortho = evd_symmetric(A)
print("reconstruct?", ok, "orthonormal Q?", ortho)


reconstruct? True orthonormal Q? True
